# Exercício 1 - Enunciado


1. O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
    Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.
    ```
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', r', t − q × t' 
    OUTPUT r, s, t
    ````
    1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.
    2. Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
        Prove que o programa nunca atinge o estado de erro
    3. Prove que a relação de Bézout $\,$ $\,a*s + b*t = r\,$ é um invariante do algoritmo.

# Exercício 1 - Solução

s é o coeficiente pelo qual 'a' multiplica
t é o coeficiente pelo qual 'b' multiplica
r é o máximo divisor comum de 'a' e 'b'

a, b > 0'\n'  
a\*s + b\*t = r = mdc(a, b)  
r = mdc(a,b)

r', s', t' representam r, s, t no próximo estado

1. Fazer função declare
2. Fazer função init
3. Fazer função trans
4.  r = 0 ou s,t overflow é estado de erro. Provar que nunca atinge estado de erro
5. Provar que a*s + b*t = r é um invariante

In [26]:
from pysmt.shortcuts import *
from pysmt.typing import INT

In [27]:
def declare(i):
    state = {}
    state['s'] = Symbol('s'+str(i),INT)
    state['t'] = Symbol('t'+str(i),INT)
    state['r'] = Symbol('r'+str(i),INT)
    state['s_prox'] = Symbol('s_prox'+str(i),INT)
    state['t_prox'] = Symbol('t_prox'+str(i),INT)
    state['r_prox'] = Symbol('r_prox'+str(i),INT)
    state['q'] = Symbol('q'+str(i),INT)
    state['pc'] = Symbol('pc'+str(i),INT)
    
    return state

In [28]:
def init(state, a, b):
    
    A = Equals(state['pc'], Int(0))
    B = GT(Int(a), Int(0))
    C = GT(Int(b), Int(0))
    
    return And(A, B, C)

In [29]:
def trans(curr, prox, a, b):
           
    t01 = And(Equals(curr['pc'], Int(0)),
              Equals(prox['pc'], Int(1)),
              
              Equals(curr['r'], Int(a)),
              Equals(curr['r_prox'], Int(b)),
              
              Equals(curr['s'], Int(1)),
              Equals(curr['s_prox'], Int(0)),
              
              Equals(curr['t'], Int(0)),
              Equals(curr['t_prox'], Int(1)),
              
              Equals(curr['s'], prox['s']),
              Equals(curr['s_prox'], prox['s_prox']),
              
              Equals(curr['r'], prox['r']),
              Equals(curr['r_prox'], prox['r_prox']),
              
              Equals(curr['t'], prox['t']),
              Equals(curr['t_prox'], prox['t_prox'])
             )
    
    t14 = And(Equals(curr['pc'], Int(1)),
              Equals(prox['pc'], Int(4)),
              
              Equals(curr['r_prox'], Int(0)),
              Equals(curr['r_prox'], prox['r_prox'])
             )
    
    t12 = And(Equals(curr['pc'], Int(1)),
              Equals(prox['pc'], Int(2)),
              
              Not(Equals(curr['r_prox'], Int(0))),
              Equals(curr['r_prox'], prox['r_prox'])
             )
    
    t23 = And(Equals(curr['pc'], Int(2)),
              Equals(prox['pc'], Int(3)),
              Equals(prox['q'], Div(curr['r'], curr['r_prox'])),
              Equals(curr['q'], prox['q'])
             )
    
    t31 = And(Equals(curr['pc'], Int(3)),
              Equals(prox['pc'], Int(1)),
              
              Equals(prox['r'], curr['r_prox']),
              Equals(curr['r'], prox['r']),
              
              Equals(prox['r_prox'], Minus(curr['r'], Times(curr['q'], curr['r_prox']))),
              Equals(curr['r_prox'], prox['r_prox']),
              
              Equals(prox['s'], curr['s_prox']), 
              Equals(curr['s'], prox['s']),

              Equals(prox['s_prox'], Minus(curr['s'], Times(curr['q'], curr['s_prox']))),
              Equals(curr['s_prox'], prox['s_prox']),
              
              Equals(prox['t'], curr['t_prox']),
              Equals(curr['t'], prox['t']),
              
              Equals(prox['t_prox'], Minus(curr['t'], Times(curr['q'], curr['t_prox']))),        
              Equals(curr['t_prox'], prox['t_prox'])
              )
    
    return Or(t01, t14, t12, t23, t31)

In [30]:
def combine(i, a, b):
    N = 10
    states = [declare(i) for i in range(N+1)]

    with Solver() as solver:
        
        solver.add_assertion(init(states[0], a, b))
        for i in range(N-1):
            solver.add_assertion(trans(states[i], states[i+1], a, b))
        solver.add_assertion(And([LT(state['pc'], Int(4)) for state in states]))
                    
        if solver.solve():
            for key, state in enumerate(states):
                print(f"> State {key}: pc = {solver.get_value(state['pc'])}\ns = {solver.get_value(state['s'])}\nt = {solver.get_value(state['t'])}\nr = {solver.get_value(state['r'])}\ns' = {solver.get_value(state['s_prox'])}\nt' = {solver.get_value(state['t_prox'])}\nr' = {solver.get_value(state['r_prox'])}")
        else:
            print("> Not feasible.")
            
combine(0, 10, 2)

> State 0: pc = 0
s = 1
t = 0
r = 10
s' = 0
t' = 1
r' = 2
> State 1: pc = 1
s = 1
t = 0
r = 10
s' = 0
t' = 1
r' = 2
> State 2: pc = 2
s = 0
t = 0
r = 1
s' = 0
t' = 0
r' = 2
> State 3: pc = 3
s = 1
t = 1
r = 1
s' = 1
t' = 1
r' = 1
> State 4: pc = 1
s = 1
t = 1
r = 1
s' = 1
t' = 1
r' = 1
> State 5: pc = 2
s = 1
t = -1
r = 0
s' = 1
t' = -1
r' = 1
> State 6: pc = 3
s = 1
t = 1
r = 1
s' = 1
t' = 1
r' = 1
> State 7: pc = 1
s = 1
t = 1
r = 1
s' = 1
t' = 1
r' = 1
> State 8: pc = 2
s = 1
t = -1
r = -1
s' = -1
t' = 1
r' = 1
> State 9: pc = 3
s = 1
t = 1
r = 1
s' = -1
t' = -1
r' = -1
> State 10: pc = 3
s = 0
t = 0
r = 0
s' = 0
t' = 0
r' = 0
